In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from glob import glob
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import copy
import pickle
import re

from utils.generic.reichlab import *

In [ ]:
predictions_pkl_filename = '/scratch/users/sansiddh/covid-modelling/2020_1022_223816/predictions_dict.pkl'
with open(predictions_pkl_filename, 'rb') as f:
    predictions_dict = pickle.load(f)

In [ ]:
date_of_submission = '2020-09-21'
comp = 'inc_case'

In [ ]:
list_of_models = get_list_of_models(date_of_submission, comp, reichlab_path='../../../covid19-forecast-hub')
df_all_submissions = process_all_submissions(list_of_models, date_of_submission, comp, reichlab_path='../../../covid19-forecast-hub')
df_gt, df_gt_loss, df_gt_loss_wk, loc_name_to_key_dict = process_gt(comp, df_all_submissions, reichlab_path='../../../covid19-forecast-hub')

In [ ]:
df_wiai_submission = format_wiai_submission(predictions_dict, df_all_submissions, loc_name_to_key_dict)
df_all_submissions = combine_wiai_subm_with_all(df_all_submissions, df_wiai_submission, comp)

In [ ]:
df_comb, df_mape, df_rank = compare_gt_pred(df_all_submissions, df_gt_loss_wk)

In [ ]:
df_mape

In [ ]:
df_rank

In [ ]:
df_rank.median(axis=1).sort_values()

In [ ]:
df_rank.mean(axis=1).sort_values()

In [ ]:
df_mape.median(axis=1).sort_values()

In [ ]:
df = pd.concat([df_mape.mean(axis=0), df_mape.std(axis=0), df_mape.loc['Wadhwani_AI', :]], axis=1)
df.columns = ['mean_mape', 'std_mape', 'wadhwani_mape']
df['z_score'] = (df['wadhwani_mape'] - df['mean_mape'])/(df['std_mape'])

In [ ]:
df

In [ ]:
fig, ax = plt.subplots(figsize=(6, 30))
sns.heatmap(df['z_score'].to_numpy().reshape(-1, 1), cmap='coolwarm', center=0, annot=True, ax=ax, yticklabels=df.index, xticklabels=['z_score'])
fig.savefig('zscore.png')

In [ ]:
from scipy.stats import zscore
import seaborn as sns

In [ ]:
df_mape

In [ ]:
zscore(df_mape.loc[:, 'Alabama'], nan_policy='omit')

In [ ]:
df_mape.loc['Wadhwani_AI', :].sort_values()

In [ ]:
df_rank.loc['Wadhwani_AI', :]

In [ ]:
df_mape.loc[:, 'Guam']